In [34]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import random
import json
import objectpath

In [2]:
stemmer = LancasterStemmer()

In [74]:
with open('./intents.json') as json_data:
    intents = json.load(json_data)

In [75]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        print(pattern)

Hi
How are you
Is anyone there?
Hello
Good day
Bye
See you later
Goodbye
Thanks
Thank you
That's helpful
What hours are you open?
What are your hours?
When are you open?
Which mopeds do you have?
What kinds of mopeds are there?
What do you rent?
Do you take credit cards?
Do you accept Mastercard?
Are you cash only?
Are you open today?
When do you open today?
What are your hours today?
today
Can we rent a moped?
I'd like to rent a moped
How does this work?


In [76]:
# get all words, classes, documents form json file 
words = []
classes = []
patterns = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern) # tokenize words
        words.extend(w) # add each to our word list
        patterns.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

In [77]:
classes

['goodbye',
 'greeting',
 'hours',
 'mopeds',
 'opentoday',
 'payments',
 'rental',
 'thanks',
 'today']

In [103]:
print(len(patterns), "patterns")
print(len(words), "words: \n", words)
print(len(classes), "classes: \n", classes)

27 patterns
48 words: 
 ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']
9 classes: 
 ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [104]:
# notice the format of patterns, which is key going forward
patterns

[(['Hi'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'),
 (['What', 'are', 'your', 'hours', '?'], 'hours'),
 (['When', 'are', 'you', 'open', '?'], 'hours'),
 (['Which', 'mopeds', 'do', 'you', 'have', '?'], 'mopeds'),
 (['What', 'kinds', 'of', 'mopeds', 'are', 'there', '?'], 'mopeds'),
 (['What', 'do', 'you', 'rent', '?'], 'mopeds'),
 (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'),
 (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'),
 (['Are', 'you', 'cash', 'only', '?'], 'payments'),
 (['Are', 'you', 'open', 'today', '?'], 'opentoday'),
 (['When', 'do', 'you', 'open', 'today', '?'], 'opentoday'),
 (['What', 'are', 'your',

<h4> Convert text data to bag of words formatted input/output arrays </h4>

In [96]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in patterns:
    # initialize our bag of words:
    bag = []
    # list of tokenized words for the pattern:
    pattern_words = doc[0]
    # stem each word:
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [102]:
print("input example: ", train_x[1])
print("output example: ", train_y[1])

input example:  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
output example:  [0, 0, 0, 0, 0, 1, 0, 0, 0]


<h4> Training and Implementing model </h4>

In [ ]:
# reset underlying graph data:
tf.reset_default_graph()

In [ ]:
# build the neural network:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation= 'softmax')
net = tflearn.regression(net)

# define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir = 'tflearn_logs')

# start training:
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

In [110]:
# save all of our data structures:
import pickle
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y':train_y}, open("training_data", "wb"))

<h3> Applying our model to independent input data </h3>

In [111]:
# restore all of our data structures
import pickle
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
# load our saved tensorflow model
model.load('./model.tflearn')

But before we can begin processing intents, we need a way to produce a bag-of-words from the user input

In [116]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bow array
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [119]:
p = bow("is your shop open today?", words, show_details=True)
print(p)

found in bag: is
found in bag: yo
found in bag: op
found in bag: today
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]


<h3> Build our reponse predictor: </h3>

In [120]:
ERROR_THRESHOLD = 0.25

def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below threshold
    results = [[i, r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if there is a classification then find matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]
                    # a pick a random response from the intent
                    return print(random.choice(i['responses']))
            
            results.pop(0)




<h3> Building our response predictor while including context: </h3>

In [122]:
context = {}
ERROR_THRESHOLD = 0.25

def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below threshold
    results = [[i, r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if there is a classification then find matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_detail: print('context:', i['context_set'])
                        context[userID] = i['context_set']
                        
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print('tag: ', i['tag'])
                 
                        # pick a random response from the intent
                        return print(random.choice(i['responses']))
                    
            results.pop(0)